# Etapa dos Projetos no pipeline do Salic

**ISSUE:** [#28](https://github.com/lappis-unb/salic-ml/issues/28)

**Etapas de Projeto:** No Salic um projeto por estar em uma das seguintes etapas:

- Admissibilidade
- Capitação e Execução
- Prestação de Contas
  - Análise do Objeto
  - Análise Financeira


**Objetivo:** Identificar quantos projetos tem em cada etapa no Salic

## Prepararando os dados

**Recarregar automaticamente os módulos**

In [1]:
%load_ext autoreload
%autoreload 2

**Importando módulos python**

In [2]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from salicml.utils.dates import Dates
from salicml.models.projects import Projects

PROJECT_ROOT = os.path.abspath(os.path.join(os.pardir, os.pardir))
DATA_FOLDER = os.path.join(PROJECT_ROOT, 'data', 'raw')

**Carregando os dados da tabela de Projetos**

In [28]:
cols = ['IdPRONAC', 'DtProtocolo', 'DtInicioExecucao', 'DtFimExecucao', 'Situacao']
projects_csv = os.path.join(DATA_FOLDER, 'projetos.csv')

projects = Projects(projects_csv)

df_projects = projects.data_frame.filter(items = cols)
display(df_projects.head())

,IdPRONAC,DtProtocolo,DtInicioExecucao,DtFimExecucao,Situacao
0,1,2008-09-12 00:00:00,2010-01-01 00:00:00,2010-12-31 00:00:00,E63
1,2,2008-09-12 00:00:00,NaN,NaN,G43
2,3,2008-09-12 00:00:00,NaN,NaN,K00
3,4,2008-09-03 00:00:00,NaN,NaN,E25
4,5,2008-09-12 00:00:00,NaN,NaN,E36


## Data na qual esse notebook foi executado

In [4]:
current_date = time.strftime("%Y-%m-%d %H:%M:%S")

display(time.strftime("%d-%m-%Y %H:%M:%S"))

'25-05-2018 10:34:29'

## Período da análise

Serão analizados os projetos criados apartir de 2009

In [30]:
year = 2009

df_projects = df_projects[df_projects['DtProtocolo'] >= '{}-01-01'.format(year)]

## Situações que indicam que um projeto foi encerrado

In [45]:
end_situations = ['A09', 'A13', 'A14', 'A16', 'A17', 'A18', 'A20', 'A23', 'A24', 'A26', 'A40', 'A41',
                  'A42', 'C09', 'D18', 'E04', 'E09', 'E36', 'E47', 'E49', 'E63', 'E64', 'E65', 'G16', 'G25',
                  'G26', 'G29', 'G30', 'G56', 'K00', 'K01', 'K02', 'L01', 'L02', 'L03', 'L04', 'L05', 
                  'L06', 'L08', 'L09', 'L10', 'L11']

## Identificando projetos na etapa de Admissibilidade

### Preparando filtros

Para identificar se um projeto está na etapa de admissibilidade será verificado as seguintes informações

1. A data de inicio de execução de um projeto deve ser posterior a data atual.
2. O projeto não pode ter uma situação que indica que ele já foi finalizado.

**Filtro 1:** A data de inicio de execução de um projeto deve ser posterior a data atual.

In [32]:
filter_start_execution_bigger_than_currentdate = df_projects['DtInicioExecucao'] > current_date

**Filtro 2:** O projeto não pode ter uma situação que indica que ele já foi finalizado.

In [33]:
filter_not_have_end_situation = ~df_projects['Situacao'].isin(end_situations)

O filtro de projetos na etapa de admissibilidade é a junção dos filtros anteriores

In [34]:
filter_admissibility = (filter_start_execution_bigger_than_currentdate &
                        filter_not_have_end_situation)

**Quantidade de projetos na fase de Admissibilidade**

In [35]:
df_projects_admissibility = df_projects[filter_admissibility]

display("Nº de projetos: {}".format(len(df_projects_admissibility)))

'Nº de projetos: 457'

## Identificando projetos na etapa de Execução/Captação

Segundo as informações passadas pelo pessoal do Ministério, a etapa de execução e captação ocorrem juntas, logo será apenas utilizado o nome "fase de execução"

### Preparando filtros

Para identificar se um projeto está na etapa de execução será verificado as seguintes informações

1. A data de inicio de execução de um projeto deve ser anterior a data atual.
2. A data de fim de execução de um projeto deve ser posterior a data atual.
3. O projeto não pode ter uma situação que indica que ele já foi finalizado.

**Filtro 1:** A data de inicio de execução de um projeto deve ser anterior a data atual.

In [36]:
filter_start_execution_before_than_currentdate = df_projects['DtInicioExecucao'] < current_date

**Filtro 2:** A data de fim de execução de um projeto deve ser posterior a data atual.

In [37]:
filter_end_execution_after_than_currentdate = df_projects['DtFimExecucao'] > current_date

**Filtro 3:** O projeto não pode ter uma situação que indica que ele já foi finalizado.

In [38]:
filter_not_have_end_situation = ~df_projects['Situacao'].isin(end_situations)

O filtro de projetos na etapa de execução é a junção dos filtros anteriores

In [39]:
filter_execution = (filter_start_execution_before_than_currentdate &
                    filter_end_execution_after_than_currentdate &
                    filter_not_have_end_situation)

**Quantidade de projetos na fase de Execução**

In [40]:
df_projects_execution = df_projects[filter_execution]

display("Nº de projetos: {}".format(len(df_projects_execution)))

# display(df_projects_execution.sort_values('DtFimExecucao'))

'Nº de projetos: 7472'

## Identificando projetos na etapa de Prestação de Contas

### Preparando filtros

Para identificar se um projeto está na etapa de prestação de contas será verificado as seguintes informações

1. A data de fim de execução de um projeto deve ser anterior a data atual.
2. O projeto não pode ter uma situação que indica que ele já foi finalizado.

**Filtro 1:** A data de fim de execução de um projeto deve ser anterior a data atual.

In [41]:
filter_end_execution_before_than_currentdate = df_projects['DtFimExecucao'] < current_date

**Filtro 3:** O projeto não pode ter uma situação que indica que ele já foi finalizado.

In [46]:
filter_not_have_end_situation = ~df_projects['Situacao'].isin(end_situations)

O filtro de projetos na etapa de execução é a junção dos filtros anteriores

In [47]:
filter_accountability = (filter_end_execution_before_than_currentdate &
                         filter_not_have_end_situation)

**Quantidade de projetos na fase de Prestação de Contas**

In [48]:
df_projects_accountability = df_projects[filter_accountability]

display("Nº de projetos: {}".format(len(df_projects_accountability)))

display(df_projects_accountability.sort_values('DtFimExecucao'))

'Nº de projetos: 17091'

,IdPRONAC,DtProtocolo,DtInicioExecucao,DtFimExecucao,Situacao
112310,112356,2009-05-08 12:20:33,2009-07-08 00:00:00,2009-01-31 00:00:00,E24
111413,111438,2009-03-26 10:02:15,2009-05-01 00:00:00,2009-02-28 00:00:00,E68
114181,114255,2009-06-29 15:39:07,2009-09-01 00:00:00,2009-03-01 00:00:00,E19
111313,111338,2009-03-13 11:02:38,2009-04-30 00:00:00,2009-04-30 00:00:00,G17
111240,111265,2009-02-27 11:43:42,2009-05-19 00:00:00,2009-06-15 00:00:00,E19
111397,111422,2009-03-24 17:14:17,2009-06-05 00:00:00,2009-06-29 00:00:00,E19
111298,111323,2009-03-10 14:57:38,2009-05-04 00:00:00,2009-06-30 00:00:00,D36
111544,111569,2009-04-06 11:59:25,2009-07-08 00:00:00,2009-07-19 00:00:00,E24
111670,111695,2009-04-14 15:32:36,2009-07-22 00:00:00,2009-07-26 00:00:00,E19
111169,111194,2009-02-11 09:27:46,2009-05-01 00:00:00,2009-07-30 00:00:00,D36
